In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)   #0.2만 테스트세트

In [9]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size= 0.2, random_state= 42) #훈련세트 중 0.2 검증세트
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state= 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))   #훈련세트 과대적합

0.9971133028626413
0.864423076923077


In [ ]:
#교차검증 이용: 훈련 세트에서 검증세트 다른 항목으로 여러번 나누어 반복 훈련 및 검증하여 안정적인 검증점수
#세 부분으로 나누면 3-fold 교차검증, 보통 5~10 많이 사용

from sklearn.model_selection import cross_validate     #교차검증 함수, 기본 5-fold
scores = cross_validate(dt, train_input, train_target)
print(scores)

# fit_time(훈련시간), score_time(검증시간), test_score 키를 가진 딕셔너리 반환

{'fit_time': array([0.00998831, 0.00899982, 0.00599623, 0.00599837, 0.00599718]), 'score_time': array([0.00299764, 0.00200057, 0.00100255, 0.002002  , 0.00100279]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score']))  #이건 split 함수로 알아서 데이터 섞여서 만들어진거 

from sklearn.model_selection import StratifiedKFold  
#데이터를 폴드로 나눌 때 각 폴드에 원래 데이터 셋에 존재하는 클래스 비율이 최대한 유지되도록 함
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) 
#StratifiedKFold() 기본값 적용: n_splits 5, shuffle false, random_state 없음
print(np.mean(scores['test_score']))

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) 
#데이터 10개 폴드로 나눔, 교차검증 전 데이터 섞음, 난수설정
scores = cross_validate(dt, train_input, train_target, cv=splitter) 
print(np.mean(scores['test_score']))

0.855300214703487
0.855300214703487


In [21]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) 
scores = cross_validate(dt, train_input, train_target, cv=splitter) 
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
#하이퍼파라미터 튜닝